In [3]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

The base16_mplrc extension is already loaded. To reload it, use:
  %reload_ext base16_mplrc
Setting plotting theme to default-light. Palette available in b16_colors


/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [6]:
import cPickle as pickle
from fuzzywuzzy import fuzz, process
from fenaroli_parser.recipe_weights import compute_all_weights, match_ingredients, compute_single_avg_weight

## ingredient words

In [7]:
with open('./data/recipes.pkl', 'r') as f:
    recipe_raw = pickle.load(f)

In [8]:
recipe_comb = {}
for f in recipe_raw:
    recipe_comb.update(f)

In [9]:
with open('./comb_recipes.pkl', 'w') as f:
    pickle.dump(recipe_comb, f)

In [12]:
cocktail_ing_lists = recipe_comb.values()
cocktail_ing_bow = [ing[0].lower() for ing in cocktail_ing_lists]
len(set(cocktail_ing_bow))

303

In [20]:
with open('./data/second_ing_flav_dict.pkl', 'r') as f:
    comp_ing_dict = pickle.load(f)

In [15]:
recipe_comb

{u'Smokin\u2019 Mary': [u'brown simple syrup',
  u'chipotle hot sauce',
  u'lapsang souchong tea',
  u'lime',
  u'Scotch',
  u'smoked black pepper',
  u'smoked paprika',
  u'vegetable juice',
  u'vodka',
  u'Liqueurs'],
 u'Grapefruit Negroni': [u'Campari',
  u'grapefruit peel',
  u'ruby red grapefruit',
  u'ruby red grapefruit juice',
  u'sweet vermouth',
  u'Gin,'],
 u'The Joy Division': [u'absinthe', u'cointreau', u'dry vermouth', u'Gin'],
 u'John the Baptist Recipe': [u'absinthe',
  u'Blume Marillen Apricot Eau de Vie',
  u'Coco Lopez',
  u'Green Spot Irish Whiskey',
  u'Marie Brizard Apry',
  u'Marie Brizard Creme de Cacao Blanc',
  u'Matcha green tea powder',
  u'Pistachio syrup*',
  u''],
 u'Harvest Highball': [u'ginger beer',
  u'lime juice',
  u'simple syrup',
  u'Vodka'],
 u'Perfect Ten Martini': [u'grapefruit twist', u'Gin'],
 u'Sunset Gold': [u'club soda',
  u'lemon juice',
  u'mint',
  u'simple syrup',
  u'Vodka'],
 u'Corpse Reviver No. 1': [u'apple brandy',
  u'calvados',


In [17]:
print recipe_comb['Mata Hari']

[u'black tea', u'cardamom', u'chai', u'cinnamon stick', u'clove', u'dried rose buds', u'ginger', u'lemon juice', u'pomegranate juice', u'simple syrup', u'sweet vermouth', u'Brandy']


In [10]:
fen_set = set(comp_ing_dict.keys())
cocktail_set = set([i.strip().lower() for sublist in cocktail_ing_lists for i in sublist if i])

NameError: name 'comp_ing_dict' is not defined

In [20]:
%page cocktail_set

In [21]:
%page fen_set

In [42]:
cocktail_ing_lists

[[u'brown simple syrup',
  u'chipotle hot sauce',
  u'lapsang souchong tea',
  u'lime',
  u'Scotch',
  u'smoked black pepper',
  u'smoked paprika',
  u'vegetable juice',
  u'vodka',
  u'Liqueurs'],
 [u'Campari',
  u'grapefruit peel',
  u'ruby red grapefruit',
  u'ruby red grapefruit juice',
  u'sweet vermouth',
  u'Gin,'],
 [u'absinthe', u'cointreau', u'dry vermouth', u'Gin'],
 [u'absinthe',
  u'Blume Marillen Apricot Eau de Vie',
  u'Coco Lopez',
  u'Green Spot Irish Whiskey',
  u'Marie Brizard Apry',
  u'Marie Brizard Creme de Cacao Blanc',
  u'Matcha green tea powder',
  u'Pistachio syrup*',
  u''],
 [u'ginger beer', u'lime juice', u'simple syrup', u'Vodka'],
 [u'grapefruit twist', u'Gin'],
 [u'club soda', u'lemon juice', u'mint', u'simple syrup', u'Vodka'],
 [u'apple brandy', u'calvados', u'sweet vermouth', u'Brandy'],
 [u'cherry', u'cr\xe8me de cacao', u'kirschwasser', u'Gin'],
 [u'cranberry juice', u'orange', u'orange juice', u'Cacha\xe7a'],
 [u'ginger beer', u'lime', u'Rum'],
 [

In [401]:
cocktail_set.intersection(fen_set)

{u'almond',
 u'apple',
 u'apple brandy',
 u'apple juice',
 u'armagnac',
 u'asparagus',
 u'bacon',
 u'banana',
 u'basil',
 u'beer',
 u'beet juice',
 u'bell pepper',
 u'berries',
 u'black pepper',
 u'black tea',
 u'blackberries',
 u'blackberry',
 u'blackcurrant',
 u'blueberries',
 u'blueberry',
 u'bourbon',
 u'brandy',
 u'butter',
 u'cardamom',
 u'carrot',
 u'celery',
 u'cherry',
 u'cherry brandy',
 u'chocolate',
 u'cider',
 u'cinnamon',
 u'clove',
 u'cloves',
 u'cocoa powder',
 u'coconut',
 u'coconut milk',
 u'coffee',
 u'cognac',
 u'concord grape',
 u'concord grapes',
 u'coriander',
 u'cranberry',
 u'cream',
 u'cucumber',
 u'cumin',
 u'dill',
 u'dried fig',
 u'egg',
 u'eggs',
 u'fennel',
 u'fig',
 u'fruit',
 u'garlic',
 u'gelatin',
 u'gin',
 u'ginger',
 u'grape',
 u'grapefruit',
 u'grapefruit juice',
 u'grapefruit peel',
 u'green tea',
 u'guava',
 u'honey',
 u'honeydew',
 u'horseradish',
 u'japanese',
 u'kiwi',
 u'kumquat',
 u'lavender',
 u'lemon',
 u'lemon juice',
 u'lemon peel',
 u'l

In [ ]:
compute_single_avg_weight()

In [283]:
match_ratios = []
for ing in comp_ing_dict.keys():
    match_ratios.append((('earl grey tea syrup', ing), ing))

In [122]:
len(cocktail_set)

936

In [270]:
cocktail_set

{u'3 cardamom pods',
 u'7up',
 u'absinthe',
 u'absinthe,',
 u'absolut smoked tea',
 u'acid phosphate',
 u'activated charcoal',
 u'agave',
 u'agave nectar',
 u'agave syrup',
 u'aged balsamic vinegar',
 u'aleppo pepper',
 u'aliz\xe9',
 u'allspice',
 u'allspice berries',
 u'allspice dram',
 u'allspice essential oil',
 u'allspice liqueur',
 u'almond',
 u'almond extract',
 u'almond milk',
 u'almond orgeat',
 u'almond syrup',
 u'aloe vera juice',
 u'amarena cherries',
 u'amarena cherry',
 u'amaretto',
 u'amaro',
 u'amaro montenegro',
 u'amaro nonino',
 u'amontillado sherry',
 u'ancho chile liqueur',
 u'ancho chile powder',
 u'ancho reyes ancho chile liqueur',
 u'anchor christmas ale',
 u'angostura bitters',
 u'anise liqueur',
 u'anisette',
 u'aperitif',
 u'aperol',
 u'apple',
 u'apple bitters',
 u'apple brandy',
 u'apple butter',
 u'apple chip',
 u'apple cider',
 u'apple cider syrup',
 u'apple juice',
 u'apple liqueur',
 u'apple slice',
 u'apple slices',
 u'applejack',
 u'apricot brandy',
 u

In [286]:
def find_matches():
    matches = []
    for ing in list(cocktail_set)[300:500]:
        matches.append(process.extractBests(ing, asc_comp_ing.keys(), scorer=fuzz.partial_token_set_ratio, score_cutoff=80))
    return matches

In [400]:
ingr = 'lemon'
print process.extractBests(ingr, asc_comp_ing.keys(), scorer=fuzz.ratio, score_cutoff=98)
print process.extractBests(ingr, asc_comp_ing.keys(), scorer=fuzz.partial_token_set_ratio, score_cutoff=0)

[(u'lemon', 100)]
[(u'lemon juice', 100), (u'and extraction products: lemongrass', 100), (u'juice (lemon', 100), (u'lemon peel', 100), (u'lemon petitgrain', 100)]


In [288]:
%page match_list

In [153]:
[match[0] for match in match_list[10]]

[u'strawberry', u'strawberry jam', u'strawberry wine']

In [38]:
sorted(match_ratios, reverse =True)

[(60, 'maple syrup'),
 (56, 'several green leaves'),
 (55, 'parsley leaves'),
 (52, 'of green tea'),
 (50, 'green tea'),
 (50, 'dry red beans'),
 (50, 'celery leaves'),
 (49, 'wintergreen leaves'),
 (48, 'partially fermented tea'),
 (48, 'bartlett pears'),
 (47, 'white grape var'),
 (47, 'tea (green)'),
 (47, 'red sage (texas sage) (s'),
 (47, 'pineapple guava (feyoa fruit'),
 (47, 'pear brandy'),
 (47, 'oriental styrax'),
 (47, 'mulberry leaves'),
 (47, 'fermented soy sauce'),
 (46, 'parsley seed oil'),
 (46, 'mackerel keta salmon'),
 (46, 'green tea cognac'),
 (46, 'elderberry fruit'),
 (46, 'barley (roasted)'),
 (46, 'a grape'),
 (45, 'wheat breads'),
 (45, 'tea tree oil'),
 (45, 'several varieties of mushrooms'),
 (45, 'prickly pear'),
 (45, 'orange trees'),
 (45, 'muscal grape'),
 (45, 'heated beans'),
 (45, 'ginger grass'),
 (45, 'celery stalk'),
 (44, 'several varieties of honey'),
 (44, 'several essential oils'),
 (44, 'natural resources'),
 (44, 'laurel leaves'),
 (44, 'heated

In [47]:
asc_comp_ing = {k.decode('utf-8'):v for k,v in comp_ing_dict.iteritems()}

In [50]:
asc_comp_ing.keys()

[u'macadamia nuts',
 u'mackerel',
 u'yellow',
 u'naranjilla fruit (solanum quitense lam )',
 u'asian',
 u'grapefruit peel oil',
 u'mandarin) to as low as 1% (palmarosa)',
 u'black chokeberry',
 u'the product obtained from natural sources is probably levorotatory',
 u'hyssop',
 u'over 300 natural sources including apples',
 u'it exhibits a positive optical rotation',
 u'mate',
 u'leaf oil (2%)',
 u'aromas:',
 u'sweet fish',
 u'several natural products: rose concentrate',
 u'buchu',
 u'the seeds of brassica napus var',
 u'reported as occurring',
 u'brandy distillate',
 u'raw lean fish',
 u'several plants',
 u'reported as the most abundant free amino acid',
 u'figs',
 u'alpinia',
 u'strawberry jam',
 u'sheep milk',
 u'free or esterified borneol has been identified',
 u'petitgrain',
 u'such as rum',
 u'cassia oil',
 u'addition to the optically inactive form',
 u'carnosine',
 u'ruta pinnata leaf oil',
 u'salvia sclarea',
 u'(vaccinium',
 u'almond',
 u'lesquerella seed meal',
 u'lavender',
 

In [ ]:
recipe_comb

In [ ]:
compute_all_weights(recipe_comb, comp_ing_dict, edge_list)

In [256]:
# def match_ingredients(recipe, comp_ing_dict):
#     sanitzed_comp_ing = {k.decode('utf-8'): v for k, v in comp_ing_dict.iteritems()}
#     matches = []
#     for ing in recipe:
#         match_list = process.extractBests(ing, sanitzed_comp_ing.keys(), scorer=fuzz.token_set_ratio, score_cutoff=85)
#         matches+= [match[0] for match in match_list]
#     return None

In [13]:
from itertools import permutations

In [14]:
with open('./backbone_edges_dict.pkl') as f:
    edge_dict = pickle.load(f)

In [15]:
# recipe_comb

In [18]:
rt = ['coffee', 'milk']

In [21]:
compute_single_avg_weight(rt, comp_ing_dict, edge_dict)

TypeError: compute_single_avg_weight() takes exactly 4 arguments (3 given)

In [17]:
35*2/()

TypeError: unsupported operand type(s) for /: 'int' and 'tuple'

In [344]:
rt = recipe_comb['Spiced Cocoa Recipe']
print len(list(permutations(rt,2)))

30


In [12]:
edge_ids = []
for ing_pair in list(permutations(match_ingredients(rt, comp_ing_dict), 2)):
    edge_ids.append(ing_pair[0] +', ' + ing_pair[1])
#     edge_ids.append(ing_pair[1] +', ' + ing_pair[0])
edge_ids

NameError: name 'permutations' is not defined

In [11]:
for edge in edge_ids:
    if edge_dict[edge] != 0:
        print edge, edge_dict[edge]

NameError: name 'edge_ids' is not defined

In [290]:
match_ingredients(rt, comp_ing_dict)

[u'maple syrup',
 u'lemon',
 u'juice',
 u'corn mint',
 u'pepper-mint oil',
 u'corn mint oil',
 u'juice (lemon',
 u'lemon juice',
 u'mint',
 u'corn mint oil (mentha arvensis l']

In [353]:
# asc_comp_ing.keys()

In [355]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 

In [365]:
recipe_sub = get_range(recipe_comb,0,10)
%prun recipe_weights = compute_all_weights(recipe_sub, comp_ing_dict, edge_dict)

In [367]:
recipe_weights

defaultdict(float,
            {u'Corpse Reviver No. 1': 6.333333333333333,
             u'Dark and Stormy ': 38.0,
             u'Grapefruit Negroni': 0.2,
             u'Harvest Highball': 4.666666666666666,
             u'John the Baptist Recipe': 1.0,
             u'Perfect Ten Martini': 0.0,
             u'Samba Breeze': 2.333333333333333,
             u'Smokin\u2019 Mary': 0.4,
             u'Sunset Gold': 0.0,
             u'The Joy Division': 0.0,
             u'The Translucent Cocktail': 0.0})

In [372]:
with open('./avg_recipe_weights.pkl', 'r') as f:
    avg_w = pickle.load(f)

In [373]:
avg_w

defaultdict(float,
            {u'Grapefruit Negroni': 0.2,
             u'Greenpoint': 0.6,
             u'Harvest Highball': 4.666666666666666,
             u'Perfect Ten Martini': 0.0,
             u'Piha Spritz Recipe': 1.857142857142857,
             u'Pumpkin &amp; Spice': 2.75,
             u'Rob Roy': 1.6666666666666665,
             u'Samba Breeze': 2.333333333333333,
             u'Sunset Gold': 0.0,
             u'The Joy Division': 0.0,
             u'The Translucent Cocktail': 0.0})

In [ ]:
with open('./new_avg_random_weights.pkl', 'r') as f:
    avg_random = pickle.load(f)